# Visualizing excited states

```{note}
- Currently pyscf, but this will be changed.
- Resolution will be improved
- To be added:
    1. More theory
    2. Electron and hole densities (e/h)
    3. Descriptors of e/h and D/A densities
```

Calculating the first two excited states of water, using SCF from PySCF and ADC from adcc:

In [ ]:
import py3Dmol
import numpy as np
import matplotlib.pyplot as plt
import adcc
from pyscf import gto, scf, tools

In [ ]:
water_geom = """
O       0.0000000000     0.1187290000     0.0000000000
H      -0.7532010000    -0.4749160000    -0.0000000000
H       0.7532010000    -0.4749160000     0.0000000000
"""

# Prepare molecule object
mol       = gto.Mole()
mol.atom  = water_geom
mol.basis = '6-31G'
mol.build()

# SCF calculation
scf_gs = scf.HF(mol)
scf_gs.kernel()

# ADC(2) calculation, two states
adc_state = adcc.adc2(scf_gs,n_singlets=2)

# Print results
print(adc_state.describe())

# Print dominant amplitudes
print(adc_state.describe_amplitudes())

Here we print the basic description of the excited states (energy, intensity, weight of double excitation amplitudes), as well as a break-down of the dominating amplitudes for each transition. From this we can start analysing the excited state by looking at the canonical MOs, but they easily becomes to diffuse to provide easy assignments. Instead we will consider two different methods by which the excitations can be more easily visualized and analysed, starting with natural transition orbitals.

### Natural transition orbitals

Natural transition orbitals (NTOs) are constructed to provide the most compact, transition-dependent transition orbitals of a specific excitation. With those, a single pair of NTOs corresponding to the hole and electron will typically dominate, and will thus provide a easily interpretable description of the excitation. For a pure HOMO to LUMO transition the NTOs would be the same as the HOMO (hole) and LUMO (electron) orbitals.

The NTOs are constructed from a singular-value decomposition (SVD) of the transition density matrix ($\mathbf{T}$):

\begin{equation}
\mathbf{UTV}^{\dagger} = \mathbf{\Lambda}
\end{equation}

where $\mathbf{U}$ and $\mathbf{V}$ are the transformation matrices correspoding to the hole and electron, respectively, and $\mathbf{\Lambda}$ is a diagonal matrix measuring the relative importance of each pair of NTOs.

Constructing the NTOs of each transition using `np.linalg.svd`. Focusing on the first excited state, and building:

```python
i = 0 # first state only

# Load transition density matrix, combine alpha and beta, and transform to numpy
tdm_ao    = adc_state.transition_dm[i].to_ao_basis()
p_tdm_tot = (tdm_ao[0] + tdm_ao[1]).to_ndarray()

# Build NTOs by singular value decomposition
u, s, v = np.linalg.svd(p_tdm_tot)

print('Dominant NTO of state {}'.format(i+1))
print('Relative importance:',np.around(s[0]/sum(s),3))
k = 0 # only look at dominant NTO pair  

# Initial
tools.cubegen.orbital(mol=mol, coeff=v[k],outfile="../img/vis/water_nto_{}_HONTO+{}.cube".format(i,k))
# Final
tools.cubegen.orbital(mol=mol, coeff=u.T[k],outfile="../img/vis/water_nto_{}_LUNTO+{}.cube".format(i,k))
```

This shows a strong dominance (0.97) of the first NTO pair.

Loading and visualizing

In [ ]:
viewer = py3Dmol.view(linked=False,viewergrid=(1,2),width=800,height=300)
i = 0 # first state only
k = 0 # only look at dominant NTO pair  

# Initial
with open("../img/vis/water_nto_{}_HONTO+{}.cube".format(i,k), "r") as f:
    cube = f.read(); f.close()
viewer.addModel(cube, "cube",viewer=(0,0)); viewer.setStyle({'stick':{}},viewer=(0,0))
viewer.addVolumetricData(cube, "cube", {"isoval": -0.05, "color": "blue", "opacity": 0.75},viewer=(0,0))
viewer.addVolumetricData(cube, "cube", {"isoval":  0.05, "color":  "red", "opacity": 0.75},viewer=(0,0))

# Final
with open("../img/vis/water_nto_{}_LUNTO+{}.cube".format(i,k), "r") as f:
    cube = f.read(); f.close() 
viewer.addModel(cube, "cube",viewer=(0,1)); viewer.setStyle({'stick':{}},viewer=(0,1))
viewer.addVolumetricData(cube, "cube", {"isoval": -0.05, "color": "blue", "opacity": 0.75},viewer=(0,1))
viewer.addVolumetricData(cube, "cube", {"isoval":  0.05, "color":  "red", "opacity": 0.75},viewer=(0,1))

viewer.rotate(-45,'x',viewer=(0,0))
viewer.rotate(-45,'x',viewer=(0,1))
viewer.show()

## Attachment and detachment densities

A different method for visualizing the transitions is to consider the attachment (A) and detachment (D) densities, which are constructed to show the density change related to an excitation. These densities, which corresponds to the hole/where electrons go from and an electron/where they go to, can then be used to look at properties such as hole and electron size, distance between the centroid (and thus level of charge-transfer), and more. For a simple HOMO-LUMO transition they would simply correspond to the square of the dominant NTOs.

D/A densities are constructed from the one-particle difference density matrix (1DDM), which is simply the difference between the on-particle density matrices of the initial and final state:

\begin{equation}
\rho_{\Delta} = \rho_f - \rho_i
\end{equation}

Diagonalizing this

\begin{equation}
\mathbf{U} \rho_{\Delta} \mathbf{U}^{\dagger} = \delta
\end{equation}

The attachment and detachment densities are constructed by considering the negative and positive eigenvalues. For a discussion on NTOs and D/A densities, see, *e.g.*, [this paper](https://onlinelibrary.wiley.com/doi/10.1002/jcc.23975).

```python
# Load transition density matrix, combine alpha and beta, and transform to numpy
p_state = adc_state.state_diffdm[i].to_ao_basis()
p_state_ao = (p_state[0] + p_state[1]).to_ndarray()

# Diagonalize the 1DDM
k, w = np.linalg.eigh(p_state_ao)
k_detach = k.copy()
k_attach = k.copy()
# Detachment: set positive eigenvalues to 0
k_detach[k > 0] = 0
# Attachment: set negative eigenvalues to 0
k_attach[k < 0] = 0 
# Back-transform with numpy
detach_ao = w @ np.diag(k_detach) @ w.T
attach_ao = w @ np.diag(k_attach) @ w.T

# Write cube-files
de = tools.cubegen.density(mol, dm=detach_ao, outfile="../img/vis/water_detachment_{}.cube".format(i))
at = tools.cubegen.density(mol, dm=attach_ao, outfile="../img/vis/water_attachment_{}.cube".format(i))
```

In [ ]:
viewer = py3Dmol.view(linked=False,viewergrid=(1,2),width=800,height=300)

# Detachment
with open("../img/vis/water_detachment_{}.cube".format(i), "r") as f:
    cube = f.read(); f.close()
viewer.addModel(cube, "cube",viewer=(0,0)); viewer.setStyle({'stick':{}},viewer=(0,0))
viewer.addVolumetricData(cube, "cube", {"isoval": -0.05, "color": "blue", "opacity": 0.75},viewer=(0,0))

# Attachment
with open("../img/vis/water_attachment_{}.cube".format(i), "r") as f:
    cube = f.read(); f.close() 
viewer.addModel(cube, "cube",viewer=(0,1)); viewer.setStyle({'stick':{}},viewer=(0,1))
viewer.addVolumetricData(cube, "cube", {"isoval":  0.05, "color":  "red", "opacity": 0.75},viewer=(0,1))

viewer.rotate(-45,'x',viewer=(0,0))
viewer.rotate(-45,'x',viewer=(0,1))
viewer.show()